# 2.1 初始神经网络

&emsp;&emsp;我们来看一个具体的神经网络示例，使用 Python 的 Keras 库来学习手写数字分类。如果你没用过 Keras 或类似的库，可能无法立刻搞懂这个例子中的全部内容。甚至你可能还没有安装Keras。没关系，下一章会详细解释这个例子中的每个步骤。因此，如果其中某些步骤看起来有些随意，或者像魔法一样，也请你不要担心。下面我们要开始了。

&emsp;&emsp;我们这里要解决的问题是，将手写数字的灰度图像（28 像素×28 像素）划分到 10 个类别中（0~9）。我们将使用 MNIST 数据集，它是机器学习领域的一个经典数据集，其历史几乎和这个领域一样长，而且已被人们深入研究。这个数据集包含 60 000 张训练图像和 10 000 张测试图
像，由美国国家标准与技术研究院（National Institute of Standards and Technology，即 MNIST 中的 NIST）在 20 世纪 80 年代收集得到。你可以将“解决”MNIST 问题看作深度学习的“Hello World”，正是用它来验证你的算法是否按预期运行。当你成为机器学习从业者后，会发现MNIST 一次又一次地出现在科学论文、博客文章等中。图 2-1 给出了 MNIST 数据集的一些样本。

**关于类和标签的说明**

&emsp;&emsp;*在机器学习中，分类问题中的某个**类别**叫作**类**（class）。数据点叫作**样本**（sample）。某个样本对应的类叫作**标签**（label）*

![](../images/2-1.png)

&emsp;&emsp;你不需要现在就尝试在计算机上运行这个例子。但如果你想这么做的话，首先需要安装Keras，安装方法见 3.3 节。<br/>
&emsp;&emsp;MNIST 数据集预先加载在 Keras 库中，其中包括 4 个 Numpy 数组。

#### 代码清单 2-1 加载 Keras 中的 MNIST 数据集

In [1]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

&emsp;&emsp;`train_images` 和 `train_labels` 组成了训练集（`training set`），模型将从这些数据中进行学习。然后在测试集（`test set`，即 `test_images` 和 `test_labels`）上对模型进行测试。

&emsp;&emsp;图像被编码为 `Numpy` 数组，而标签是数字数组，取值范围为 0~9。图像和标签一一对应。

&emsp;&emsp;我们来看一下训练数据：

In [2]:
train_images.shape

(60000, 28, 28)

In [3]:
len(train_labels)

60000

In [4]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

&emsp;&emsp;下面是测试数据：

In [5]:
test_images.shape

(10000, 28, 28)

In [6]:
len(test_labels)

10000

In [7]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

&emsp;&emsp;接下来的工作流程如下：首先，将训练数据（`train_images` 和 `train_labels`）输入神经网络；其次，网络学习将图像和标签关联在一起；最后，网络对 `test_images` 生成预测，而我们将验证这些预测与 `test_labels` 中的标签是否匹配。

&emsp;&emsp;下面我们来构建网络。再说一遍，你现在不需要理解这个例子的全部内容。

#### 代码清单 2-2 网络架构

In [8]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation="softmax"))

&emsp;&emsp;神经网络的核心组件是**层**（`layer`），他是一种数据处理模块，你可以将它看成数据过滤器。进去一些数据，出来的数据变得更加有用。具体来说，层从输入数据中提取**表示**——我们期望这种表示有助于解决手头的问题。大多数深度学习都是将简单的层链接起来，从而实现渐进式的**数据蒸馏**（`data distillation`）。深度学习模型就像是数据处理的筛子，包含一系列越来越精细的数据过滤器（即层）

&emsp;&emsp;本例中的网络包含 2 个 `Dense` 层，它们是密集连接（也叫**全连接**）的神经层。第二层（也是最后一层）是一个 10 路 `softmax` 层，它将返回一个由 10 个概率值（总和为 1）组成的数组。每个概率值表示当前数字图像属于 10 个数字类别中某一个的概率。

&emsp;&emsp;要想训练网络，我们还需要选择**编译**（compile）步骤的三个参数。
- **损失函数**（`loss function`）：网络如何衡量在训练数据上的性能，即网络如何朝着正确的方向前进。
- **优化器**（`optimizer`）：基于训练数据和损失函数来更新网络的机制。
- **在训练和测试过程中需要监控的指标**（`metric`）：本例只关心精度，即正确分类的图像所占的比例。

&emsp;&emsp;后续两章会详细解释损失函数和优化器的确切用途。

#### 代码清单 2-3 编译步骤

In [9]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

&emsp;&emsp;在开始训练之前，我们将对数据进行预处理，将其变换为网络要求的形状，并缩放到所有值都在 [0, 1] 区间。比如，之前训练图像保存在一个 uint8 类型的数组中，其形状为(60000, 28, 28)，取值区间为 [0, 255]。我们需要将其变换为一个 float32 数组，其形状为 (60000, 28 * 28)，取值范围为 0~1。

#### 代码清单 2-4 准备图像数据

In [10]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255.0

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255.0

&emsp;&emsp;我们还需要对标签进行分类编码，第 3 章将会对这一步骤进行解释。

#### 代码清单 2-5 准备标签

In [11]:
from keras.utils.np_utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

&emsp;&emsp;现在我们准备开始训练网络，在 Keras 中这一步是通过调用网络的 fit 方法来完成的——我们在训练数据上拟合（fit）模型。

In [12]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 3s 5ms/step - loss: 0.2579 - accuracy: 0.9258
Epoch 2/5
469/469 [==============================] - 2s 5ms/step - loss: 0.1052 - accuracy: 0.9689
Epoch 3/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0694 - accuracy: 0.9795
Epoch 4/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0506 - accuracy: 0.9849
Epoch 5/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0387 - accuracy: 0.9883


&emsp;&emsp;训练过程中显示了两个数字：一个是网络在训练数据上的损失（`loss`），另一个是网络在训练数据上的精度（`acc`）

&emsp;&emsp;我们很快就在训练数据上达到了 0.989（98.9%）的精度。现在我们来检查一下模型在测试集上的性能。

In [18]:
test_loss, test_acc = network.evaluate(test_images, test_labels)
print("test_acc:", test_acc)

313/313 [==============================] - 0s 1ms/step - loss: 0.0655 - accuracy: 0.9819
test_acc: 0.9818999767303467


&emsp;&emsp;测试集精度为 97.8%，比训练集精度低不少。训练精度和测试精度之间的这种差距是过拟合（`overfit`）造成的。过拟合是指机器学习模型在新数据上的性能往往比在训练数据上要差，它是第 3 章的核心主题。

&emsp;&emsp;第一个例子到这里就结束了。你刚刚看到了如何构建和训练一个神经网络，用不到 20 行的Python 代码对手写数字进行分类。下一章会详细介绍这个例子中的每一个步骤，并讲解其背后的原理。接下来你将要学到张量（输入网络的数据存储对象）、张量运算（层的组成要素）和梯度下降（可以让网络从训练样本中进行学习）